In [ ]:
import json
from dataclasses import dataclass
from html import unescape

from google.colab import files

In [ ]:
def twinery_html_to_episode(html_text):
    story_data = html_text.split('<tw-storydata')[1].split('</tw-storydata>')[0]
    story_data = unescape(story_data)

    episode_title = story_data.split('name="')[1].split('"')[0]

    pages = list()
    for passage_data in story_data.split('<tw-passagedata')[1:]:
        page_id = passage_data.split('name="')[1].split('"')[0].strip()
            
        passage_body = passage_data.split('">')[1].split('</tw-passagedata>')[0]
        passage_lines = [line.strip() for line in passage_body.split('\n')]

        body_lines = [
            line for line in passage_lines
            if not line.startswith('[[') and not line.endswith(']]')
        ]
        
        action_lines = [
            line.strip('[]') for line in passage_lines
            if line.startswith('[[') and line.endswith(']]')
        ]
        actions = [{
            'title': line.split('->')[0].strip(),
            'linkedPageId': line.split('->')[1].strip(),
        } for line in action_lines]
        
        page = {
            'id': page_id,
            'body': {'lines': body_lines},
            'actions': actions,
            'isEnd': len(actions) == 0,
        }
        pages.append(page)

    episode = {
        'id': None,  # TODO
        'title': episode_title,
        'intro': None,  # TODO
        'pageList': pages,
        'startPageId': None,  # TODO
    }
    return episode

In [ ]:
uploaded = files.upload()

content = next(iter(uploaded.values()))
html_text = content.decode('utf-8')

episode = twinery_html_to_episode(html_text)
print(json.dumps(episode, indent=2, ensure_ascii=False))